In [ ]:
from keras.models import Sequential,load_model
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator


import sys
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
def define_model():
  #1 layer
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
 
  #2 layer
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))
  
  #3 layer
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.4))
 
  #4 layer
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
 
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
#data preprocessing(whiten)
mean = [0,0,0]
std = [0,0,0]
newX_train = np.ones(x_train.shape)
newX_test = np.ones(x_test.shape)

#train set에 있는 데이터로 mean과 std를 구함
for i in range(3): #channel = 3
  mean[i] = np.mean(x_train[ : , : , : , i])
  std[i] = np.std(x_train[ : , : , : ,i])

#정규화하기 N(0,1)
for i in range(3):
  newX_train[ : , : , : , i] = x_train[ : , : , : ,i] - mean[i]
  newX_train[ : , : , : , i] = newX_train[ : , : , : ,i] / std[i]
  newX_test[ : , : , : , i] = x_test[ : , : , : ,i] - mean[i]
  newX_test[ : , : , : , i] = newX_test[ : , : , : ,i] / std[i]

X_train = newX_train
X_test = newX_test
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
#run the test harness for evaluating a model
# 1. define model -> upper
# 2. preprocessing data -> upper
# 3. make model
model = define_model()
# 4. fit model
model.fit(X_train, Y_train, epochs=100, batch_size=64, verbose=0)
# 5. save model
model.save('final_model.h5')
# 6. load model
trained_model = load_model('final_model.h5')
# 7. evaluate trained model on test dataset
_, acc = trained_model.evaluate(X_test, Y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

KeyboardInterrupt: ignored

In [ ]:
"""
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(32, 32))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 32, 32, 3)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img

# load an image and predict the class
def run_example():
	# load the image
	img = load_image('sample_image.png')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	result = model.predict_classes(img)
	print(result[0])

# entry point, run the example
run_example()
"""

# evaluate data from X_test (including Y_test)
# 1.use softmax  2.without softmax